# CLIPDraw with 3d strokes

Synthesize (3d) drawings to match a text prompt.

This places the output in created directories tmp/ and output/, but doesn't display the results inside the notebook itself.

To make a rotating view, after training for as long as you like, run the cell under the Rotating View header

This version is set up to use a different prompt for different angles, though you can set all the prompts identical to avoid this effect.

## Imports and setup

In [6]:
# To use in Colab, probably need this...

#!pip install clip

In [4]:
import os
import io
import numpy as np
import glob
import subprocess
from math import *
import clip

from PIL import Image
import random
#import ttools.modules
import argparse

import torch
import torch.nn.functional as F
from torchvision import transforms
import torchvision

!mkdir tmp
!mkdir output


mkdir: cannot create directory ‘tmp’: File exists
mkdir: cannot create directory ‘output’: File exists


In [7]:
#@title Load CLIP {vertical-output: true}

# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Load the model
device = torch.device('cuda')
model, preprocess = clip.load('ViT-B/32', device, jit=False)

nouns = "aardvark abyssinian accelerator accordion account accountant acknowledgment acoustic acrylic act action active activity actor actress adapter addition address adjustment adult advantage advertisement advice afghanistan africa aftermath afternoon aftershave afterthought age agenda agreement air airbus airmail airplane airport airship alarm albatross alcohol algebra algeria alibi alley alligator alloy almanac alphabet alto aluminium aluminum ambulance america amount amusement anatomy anethesiologist anger angle angora animal anime ankle answer ant antarctica anteater antelope anthony anthropology apartment apology apparatus apparel appeal appendix apple appliance approval april aquarius arch archaeology archeology archer architecture area argentina argument aries arithmetic arm armadillo armchair armenian army arrow art ash ashtray asia asparagus asphalt asterisk astronomy athlete atm atom attack attempt attention attic attraction august aunt australia australian author authorisation authority authorization avenue babies baboon baby back backbone bacon badge badger bag bagel bagpipe bail bait baker bakery balance balinese ball balloon bamboo banana band bandana bangladesh bangle banjo bank bankbook banker bar barbara barber barge baritone barometer base baseball basement basin basket basketball bass bassoon bat bath bathroom bathtub battery battle bay beach bead beam bean bear beard beast beat beautician beauty beaver bed bedroom bee beech beef beer beet beetle beggar beginner begonia behavior belgian belief believe bell belt bench bengal beret berry bestseller betty bibliography bicycle bike bill billboard biology biplane birch bird birth birthday bit bite black bladder blade blanket blinker blizzard block blood blouse blow blowgun blue board boat bobcat body bolt bomb bomber bone bongo bonsai book bookcase booklet boot border botany bottle bottom boundary bow bowl bowling box boy bra brace bracket brain brake branch brand brandy brass brazil bread break breakfast breath brian brick bridge british broccoli brochure broker bronze brother brother-in-law brow brown brush bubble bucket budget buffer buffet bugle building bulb bull bulldozer bumper bun burglar burma burn burst bus bush business butane butcher butter button buzzard cabbage cabinet cable cactus cafe cake calculator calculus calendar calf call camel camera camp can canada canadian cancer candle cannon canoe canvas cap capital cappelletti capricorn captain caption car caravan carbon card cardboard cardigan care carnation carol carp carpenter carriage carrot cart cartoon case cast castanet cat catamaran caterpillar cathedral catsup cattle cauliflower cause caution cave c-clamp cd ceiling celery celeste cell cellar cello celsius cement cemetery cent centimeter century ceramic cereal certification chain chair chalk chance change channel character chard charles chauffeur check cheek cheese cheetah chef chemistry cheque cherries cherry chess chest chick chicken chicory chief child children chill chime chimpanzee chin china chinese chive chocolate chord christmas christopher chronometer church cicada cinema circle circulation cirrus citizenship city clam clarinet class claus clave clef clerk click client climb clipper cloakroom clock close closet cloth cloud cloudy clover club clutch coach coal coast coat cobweb cockroach cocktail cocoa cod coffee coil coin coke cold collar college collision colombia colon colony color colt column columnist comb comfort comic comma command commission committee community company comparison competition competitor composer composition computer condition condor cone confirmation conga congo conifer connection consonant continent control cook cooking copper copy copyright cord cork cormorant corn cornet correspondent cost cotton couch cougar cough country course court cousin cover cow cowbell crab crack cracker craftsman crate crawdad crayfish crayon cream creator creature credit creditor creek crib cricket crime criminal crocodile crocus croissant crook crop cross crow crowd crown crush cry cub cuban cucumber cultivator cup cupboard cupcake curler currency current curtain curve cushion custard customer cut cuticle cycle cyclone cylinder cymbal dad daffodil dahlia daisy damage dance dancer danger daniel dash dashboard database date daughter david day dead deadline deal death deborah debt debtor decade december decimal decision decrease dedication deer defense deficit degree delete delivery den denim dentist deodorant department deposit description desert design desire desk dessert destruction detail detective development dew diamond diaphragm dibble dictionary dietician difference digestion digger digital dill dime dimple dinghy dinner dinosaur diploma dipstick direction dirt disadvantage discovery discussion disease disgust dish distance distribution distributor diving division divorced dock doctor dog dogsled doll dollar dolphin domain donald donkey donna door dorothy double doubt downtown dragon dragonfly drain drake drama draw drawbridge drawer dream dredger dress dresser dressing drill drink drive driver driving drizzle drop drug drum dry dryer duck duckling dugout dungeon dust eagle ear earth earthquake ease east edge edger editor editorial education edward eel effect egg eggnog eggplant egypt eight elbow element elephant elizabeth ellipse emery employee employer encyclopedia end enemy energy engine engineer engineering english enquiry entrance environment epoch epoxy equinox equipment era error estimate ethernet ethiopia euphonium europe evening event examination example exchange exclamation exhaust ex-husband existence expansion experience expert explanation ex-wife eye eyebrow eyelash eyeliner face facilities fact factory fahrenheit fairies fall family fan fang farm farmer fat father father-in-law faucet fear feast feather feature february fedelini feedback feeling feet felony female fender ferry ferryboat fertilizer fiber fiberglass fibre fiction field fifth fight fighter file find fine finger fir fire fired fireman fireplace firewall fish fisherman flag flame flare flat flavor flax flesh flight flock flood floor flower flugelhorn flute fly foam fog fold font food foot football footnote force forecast forehead forest forgery fork form format fortnight foundation fountain fowl fox foxglove fragrance frame france freckle freeze freezer freighter french freon friction friday fridge friend frog front frost frown fruit fuel fur furniture galley gallon game gander garage garden garlic gas gasoline gate gateway gauge gazelle gear gearshift geese gemini gender geography geology geometry george geranium german germany ghana ghost giant giraffe girdle girl gladiolus glass glider gliding glockenspiel glove glue goal goat gold goldfish golf gondola gong good-bye goose gore-tex gorilla gosling government governor grade grain gram granddaughter grandfather grandmother grandson grape graphic grass grasshopper gray grease great-grandfather great-grandmother greece greek green grenade grey grill grip ground group grouse growth guarantee guatemalan guide guilty guitar gum gun gym gymnast hacksaw hail hair haircut half-brother half-sister halibut hall hallway hamburger hammer hamster hand handball handicap handle handsaw harbor hardboard hardcover hardhat hardware harmonica harmony harp hat hate hawk head headlight headline health hearing heart heat heaven hedge height helen helicopter helium hell helmet help hemp hen heron herring hexagon hill himalayan hip hippopotamus history hobbies hockey hoe hole holiday home honey hood hook hope horn horse hose hospital hot hour hourglass house hovercraft hub hubcap humidity humor hurricane hyacinth hydrant hydrofoil hydrogen hyena hygienic ice icebreaker icicle icon idea ikebana illegal imprisonment improvement impulse inch income increase index india indonesia industry ink innocent input insect instruction instrument insulation insurance interactive interest internet interviewer intestine invention inventory invoice iran iraq iris iron island israel italian italy jacket jaguar jail jam james january japan japanese jar jasmine jason jaw jeans jeep jeff jelly jellyfish jennifer jet jewel jogging john join joke joseph journey judge judo juice july jumbo jump jumper june jury justice jute kale kamikaze kangaroo karate karen kayak kendo kenneth kenya ketchup kettle kettledrum kevin key keyboard keyboarding kick kidney kilogram kilometer kimberly kiss kitchen kite kitten kitty knee knickers knife knight knot knowledge kohlrabi korean laborer lace ladybug lake lamb lamp lan land landmine language larch lasagna latency latex lathe laugh laundry laura law lawyer layer lead leaf learning leather leek leg legal lemonade lentil leo leopard letter lettuce level libra library license lier lift light lightning lilac lily limit linda line linen link lion lip lipstick liquid liquor lisa list literature litter liver lizard llama loaf loan lobster lock locket locust look loss lotion love low lumber lunch lunchroom lung lunge lute luttuce lycra lynx lyocell lyre lyric macaroni machine macrame magazine magic magician maid mail mailbox mailman makeup malaysia male mall mallet man manager mandolin manicure manx map maple maraca marble march margaret margin maria marimba mark mark market married mary mascara mask mass match math mattock may mayonnaise meal measure meat mechanic medicine meeting melody memory men menu mercury message metal meteorology meter methane mexican mexico mice michael michelle microwave middle mile milk milkshake millennium millimeter millisecond mimosa mind mine minibus mini-skirt minister mint minute mirror missile mist mistake mitten moat modem mole mom monday money monkey month moon morning morocco mosque mosquito mother mother-in-law motion motorboat motorcycle mountain mouse moustache mouth move multi-hop multimedia muscle museum music musician mustard myanmar nail name nancy napkin narcissus nation neck need needle neon nepal nephew nerve nest net network news newsprint newsstand nic nickel niece nigeria night nitrogen node noise noodle north north america north korea norwegian nose note notebook notify novel november number numeric nurse nut nylon oak oatmeal objective oboe observation occupation ocean ocelot octagon octave october octopus odometer offence offer office oil okra olive onion open opera operation ophthalmologist opinion option orange orchestra orchid order organ organisation organization ornament ostrich otter ounce output outrigger oval oven overcoat owl owner ox oxygen oyster package packet page pail pain paint pair pajama pakistan palm pamphlet pan pancake pancreas panda pansy panther panties pantry pants panty pantyhose paper paperback parade parallelogram parcel parent parentheses park parrot parsnip part particle partner partridge party passbook passenger passive pasta paste pastor pastry patch path patient patio patricia paul payment pea peace peak peanut pear pedestrian pediatrician peen peer-to-peer pelican pen penalty pencil pendulum pentagon peony pepper perch perfume period periodical peripheral permission persian person peru pest pet pharmacist pheasant philippines philosophy phone physician piano piccolo pickle picture pie pig pigeon pike pillow pilot pimple pin pine ping pink pint pipe pisces pizza place plain plane planet plant plantation plaster plasterboard plastic plate platinum play playground playroom pleasure plier plot plough plow plywood pocket poet point poison poland police policeman polish politician pollution polo polyester pond popcorn poppy population porch porcupine port porter position possibility postage postbox pot potato poultry pound powder power precipitation preface prepared pressure price priest print printer prison probation process processing produce product production professor profit promotion propane property prose prosecution protest protocol pruner psychiatrist psychology ptarmigan puffin pull puma pump pumpkin punch punishment puppy purchase purple purpose push pvc pyjama pyramid quail quality quart quarter quartz queen question quicksand quiet quill quilt quince quit quiver quotation rabbi rabbit racing radar radiator radio radish raft rail railway rain rainbow raincoat rainstorm rake ramie random range rat rate raven ravioli ray rayon reaction reading reason receipt recess record recorder rectangle red reduction refrigerator refund regret reindeer relation relative religion relish reminder repair replace report representative request resolution respect responsibility rest restaurant result retailer revolve revolver reward rhinoceros rhythm rice richard riddle rifle ring rise risk river riverbed road roadway roast robert robin rock rocket rod roll romania romanian ronald roof room rooster root rose rotate route router rowboat rub rubber rugby rule run russia russian rutabaga ruth sack sagittarius sail sailboat sailor salad salary sale salesman salmon salt sampan samurai sand sandra sandwich santa sarah sardine satin saturday sauce saudi arabia sausage save saw saxophone scale scallion scanner scarecrow scarf scene scent schedule school science scissors scooter scorpio scorpion scraper screen screw screwdriver sea seagull seal seaplane search seashore season seat second secretary secure security seed seeder segment select selection self semicircle semicolon sense sentence separated september servant server session sex shade shadow shake shallot shame shampoo shape share shark sharon shears sheep sheet shelf shell shield shingle ship shirt shock shoe shoemaker shop shorts shoulder shovel show shrimp shrine siamese siberian side sideboard sidecar sidewalk sign signature silica silk silver sing singer single sink sister sister-in-law size skate skiing skill skin skirt sky slash slave sled sleep sleet slice slime slip slipper slope smash smell smile smoke snail snake sneeze snow snowboarding snowflake snowman snowplow snowstorm soap soccer society sociology sock soda sofa softball softdrink software soil soldier son song soprano sort sound soup sousaphone south africa south america south korea soy soybean space spade spaghetti spain spandex spark sparrow spear specialist speedboat sphere sphynx spider spike spinach spleen sponge spoon spot spring sprout spruce spy square squash squid squirrel stage staircase stamp star start starter state statement station statistic steam steel stem step step-aunt step-brother stepdaughter step-daughter step-father step-grandfather step-grandmother stepmother step-mother step-sister stepson step-son step-uncle steven stew stick stinger stitch stock stocking stomach stone stool stop stopsign stopwatch store storm story stove stranger straw stream street streetcar stretch string structure study sturgeon submarine substance subway success sudan suede sugar suggestion suit summer sun sunday sundial sunflower sunshine supermarket supply support surfboard surgeon surname surprise susan sushi swallow swamp swan sweater sweatshirt sweatshop swedish sweets swim swimming swing swiss switch sword swordfish sycamore syria syrup system table tablecloth tabletop tachometer tadpole tail tailor taiwan talk tank tanker tanzania target taste taurus tax taxi taxicab tea teacher teaching team technician teeth television teller temper temperature temple tempo tendency tennis tenor tent territory test text textbook texture thailand theater theory thermometer thing thistle thomas thought thread thrill throat throne thumb thunder thunderstorm thursday ticket tie tiger tights tile timbale time timer timpani tin tip tire titanium title toad toast toe toenail toilet tomato tom-tom ton tongue tooth toothbrush toothpaste top tornado tortellini tortoise touch tower town toy tractor trade traffic trail train tramp transaction transmission transport trapezoid tray treatment tree trial triangle trick trigonometry trip trombone trouble trousers trout trowel truck trumpet trunk t-shirt tsunami tub tuba tuesday tugboat tulip tuna tune turkey turkey turkish turn turnip turnover turret turtle tv twig twilight twine twist typhoon tyvek uganda ukraine ukrainian umbrella uncle underclothes underpants undershirt underwear unit united kingdom unshielded use utensil uzbekistan vacation vacuum valley value van vase vault vegetable vegetarian veil vein velvet venezuela venezuelan verdict vermicelli verse vessel vest veterinarian vibraphone vietnam view vinyl viola violet violin virgo viscose vise vision visitor voice volcano volleyball voyage vulture waiter waitress walk wall wallaby wallet walrus war warm wash washer wasp waste watch watchmaker water waterfall wave wax way wealth weapon weasel weather wedge wednesday weed weeder week weight whale wheel whip whiskey whistle white wholesaler whorl wilderness william willow wind windchime window windscreen windshield wine wing winter wire wish witch withdrawal witness wolf woman women wood wool woolen word work workshop worm wound wrecker wren wrench wrinkle wrist writer xylophone yacht yak yam yard yarn year yellow yew yogurt yoke yugoslavian zebra zephyr zinc zipper zone zoo zoology"
nouns = nouns.split(" ")
noun_prompts = ["a drawing of a " + x for x in nouns]

# Calculate features
with torch.no_grad():
    nouns_features = model.encode_text(torch.cat([clip.tokenize(noun_prompts).to(device)]))
print(nouns_features.shape, nouns_features.dtype)


torch.Size([2343, 512]) torch.float16


### Differentiable 3d stroke renderer

In [8]:
def get_blobs(cam_pos, cam_targ, xyz, rgba, width, density=2):
    xx,yy = np.meshgrid(np.arange(canvas_width), np.arange(canvas_height))
    canv_coords = np.concatenate([yy[:,:,np.newaxis], xx[:,:,np.newaxis]], axis=2)
    canv_coords = torch.FloatTensor(canv_coords).to(device)
    
    up = torch.FloatTensor([0,1,0]).to(device)
    
    cam_u = F.normalize(cam_targ-cam_pos, dim=0)
    cam_v = torch.cross(cam_u, up)
    cam_w = torch.cross(cam_u, cam_v)
    
    pos = xyz - cam_pos.unsqueeze(0)
    pos_u = torch.sum(pos*cam_u.unsqueeze(0), 1)
    pos_v = torch.sum(pos*cam_v.unsqueeze(0), 1)
    pos_w = torch.sum(pos*cam_w.unsqueeze(0), 1)
    
    pos_x = canvas_width * pos_w / (1e-4 + pos_u) + canvas_width // 2
    pos_y = canvas_width * pos_v / (1e-4 + pos_u) + canvas_height // 2
    pos_xy = torch.cat([pos_x.unsqueeze(1), pos_y.unsqueeze(1)], 1)
    modified_w = canvas_width * width / (1e-4 + pos_u)
    
    blobs_rgb = []
    blobs_alpha = []
    blob_depths = []
    blob_offsets = []
    
    for i in range(xyz.shape[0]-1):
        dist = torch.sqrt(torch.sum( (pos_xy[i] - pos_xy[i+1])**2 ))
        wbar = (modified_w[i] + modified_w[i+1])*0.5
        steps = int((density * dist / (1 + wbar)).cpu().detach().item()) + 1
        if steps<2:
            steps = 2
        if steps>15:
            steps = 15
        
        for j in range(steps):
            s = j/(steps-1)
            
            p = pos_xy[i]*(1-s) + pos_xy[i+1]*s
            c = rgba[i]*(1-s) + rgba[i+1]*s
            w = modified_w[i]*(1-s) + modified_w[i+1]*s
            d = pos_u[i]*(1-s) + pos_u[i+1]*s
            
            if d>0.05: # Culling                
                bounds = 2*int(2*w.cpu().detach().item())+2
                offset = (p.cpu().detach().numpy()-bounds//2).astype(np.int32)
                if offset[0]<0:
                    offset[0]=0
                if offset[1]<0:
                    offset[1]=0
                
                cc = canv_coords[offset[0]:offset[0]+bounds, offset[1]:offset[1]+bounds]
                
                r2 = torch.sum( (cc-p.unsqueeze(0).unsqueeze(1))**2, 2)/w**2

                r2 = r2.unsqueeze(2)
                
                blob_a = c[3:4].unsqueeze(0).unsqueeze(1) * torch.exp(-r2**2)
                
                blob_offsets.append(offset)
                blob_depths.append(d.cpu().detach().item())                
                blobs_rgb.append(c[:3].unsqueeze(1))
                blobs_alpha.append(blob_a)
    
    return blob_depths, blobs_rgb, blobs_alpha, blob_offsets

def bounded_composition(canvas, offset, rgb, alpha):
    # Need to split out the affected square, then concatenate everything back
    mod_part = canvas[offset[0]:offset[0]+alpha.shape[0], offset[1]:offset[1]+alpha.shape[1]]
    
    mod_part = (1-alpha)*mod_part + alpha * rgb.unsqueeze(0).unsqueeze(1)
    
    pre_x = canvas[:offset[0], :, :]
    
    top_bit = canvas[offset[0]:offset[0]+alpha.shape[0], :offset[1], :]
    bottom_bit = canvas[offset[0]:offset[0]+alpha.shape[0], offset[1]+alpha.shape[1]:, :]
    
    post_x = canvas[offset[0]+alpha.shape[0]:, :, :]
    
    central_column = torch.cat([top_bit, mod_part, bottom_bit], 1)
    
    canvas = torch.cat([pre_x, central_column, post_x], 0)
    
    return canvas
    
def compose_blobs(depths, rgbs, alphas, offsets):
    canvas = torch.ones(canvas_width, canvas_height, 3).to(device)
    
    idx = np.argsort(-np.array(depths))
    rgbs = torch.cat(rgbs, 1)   
    
    for i in range(rgbs.shape[1]):
        canvas = bounded_composition(canvas, offsets[idx[i]], rgbs[:,idx[i]], alphas[idx[i]])
        
    return canvas

def render(cam_pos, cam_targ, strokes):
    depths = []
    rgbs = []
    alphas = []
    offsets = []
    for s in strokes:
        bd,br,ba,bo = get_blobs(cam_pos, cam_targ, s[0], s[1], s[2])
        depths = depths + bd
        rgbs = rgbs + br
        alphas = alphas + ba
        offsets = offsets + bo

    img = compose_blobs(depths, rgbs, alphas, offsets).view(canvas_width, canvas_height, 3)
    
    del depths
    del rgbs
    del alphas
    del offsets
    
    return img

## Enter prompts here!

In [5]:
#@title Curve Optimizer {vertical-output: true}

prompt1 = "The silhouette of a rabbit"
text_input1 = clip.tokenize(prompt1).to(device)

prompt2 = "A vase containing flowers"
text_input2 = clip.tokenize(prompt2).to(device)

prompt3 = "A waluigi"
text_input3 = clip.tokenize(prompt3).to(device)

# Calculate features
with torch.no_grad():
    text_features1 = model.encode_text(text_input1)
    text_features2 = model.encode_text(text_input2)
    text_features3 = model.encode_text(text_input3)

gamma = 1.0

args = lambda: None
args.num_paths = 200
args.max_width = 0.5
args.num_iter = 500
args.use_blob = False
args.use_lpips_loss = False

# Use GPU if available
device = torch.device('cuda')

canvas_width, canvas_height = 224, 224
num_paths = args.num_paths
max_width = args.max_width

augment_trans = transforms.Compose([
    transforms.RandomPerspective(fill=1, p=1),
    transforms.RandomResizedCrop(224, scale=(0.7,0.9)),
])

## Initialize strokes

In [6]:
points_vars = []
width_vars = []
color_vars = []

strokes = []

for i in range(num_paths):
    xyz = []
    rgba = []
    ws = []
    
    p0 = 2*np.random.rand(3)-1
    w0 = np.random.rand()*0.1+0.02
    rgb = np.random.rand(4)
    
    for j in range(4):
        xyz.append(p0)
        ws.append(w0)
        rgba.append(rgb)
        
        p0 = p0 + 0.3 * (2*np.random.rand(3)-1)
    
    xyz = torch.FloatTensor(np.array(xyz)).to(device)
    xyz.requires_grad = True
    points_vars.append(xyz)
    
    rgba = torch.FloatTensor(np.array(rgba)).to(device)
    rgba.requires_grad = True
    color_vars.append(rgba)

    ws = torch.FloatTensor(np.array(ws)).to(device)
    ws.requires_grad = True
    width_vars.append(ws)
    
    strokes.append([xyz, rgba, ws])

print("Strokes initialized")
# Optimize
points_optim = torch.optim.Adam(points_vars, lr=0.05)
width_optim = torch.optim.Adam(width_vars, lr=0.02)
color_optim = torch.optim.Adam(color_vars, lr=0.1)

Strokes initialized


## Train strokes

In [7]:
# Adam iterations.
for t in range(args.num_iter):
    points_optim.zero_grad()
    width_optim.zero_grad()
    color_optim.zero_grad()
    
    if t==200: # Some learning rate decay to help a drawing settle
        for p in points_optim.param_groups:
            p['lr'] *= 0.2

        for p in width_optim.param_groups:
            p['lr'] *= 0.2

        for p in color_optim.param_groups:
            p['lr'] *= 0.2    
    
    for n in range(4):
        if n==3: # Use a fixed reference for at least one, for visualization
            cam_pos = np.array([1, 0.1, 0])
        else:
            cam_pos = np.array([np.random.randn(), 0.2*np.random.rand(), np.random.randn()])
        
        theta = atan2(cam_pos[2], cam_pos[0])
        
        theta = (theta+2*3.14159) % (2*3.14159)
        
        if theta>0 and theta<=2*3.14159/3:
            features = text_features1
        elif theta<=4*3.14159/3:
            features = text_features2
        else:
            features = text_features3
            
        cam_pos = F.normalize(torch.FloatTensor(cam_pos).to(device), dim=0)*3
        cam_targ = torch.zeros(3).to(device)

        img = render(cam_pos, cam_targ, strokes)
        
        # Save the intermediate render.
        if n==3:
            im = Image.fromarray(np.clip(255*img.cpu().detach().numpy(),0,255).astype(np.uint8))
            im.save("tmp/inter_%.6d.png" % t)
        
        # Convert img from HWC to NCHW
        img = img.unsqueeze(0)
        img = img.permute(0, 3, 1, 2) # NHWC -> NCHW

        loss = 0
        for m in range(8):
            img_aug = augment_trans(img)
            image_features = model.encode_image(img_aug)
            loss -= torch.cosine_similarity(features, image_features, dim=1)/8.0
            

        # Backpropagate the gradients.
        loss.backward()

    # Take a gradient descent step.
    points_optim.step()
    width_optim.step()
    color_optim.step()
    for s in strokes:
        s[2].data.clamp_(0.02, max_width)
        s[1].data.clamp_(0.0, 1.0)
        s[0].data.clamp_(-1.0, 1.0)
    
    if t % 1 == 0:
        # show_img(torch.cat([img.detach(), img_aug.detach()], axis=3).cpu().numpy()[0])
        print('render loss:', loss.item())
        print('iteration:', t)
        

render loss: -0.24462890625
iteration: 0
render loss: -0.2548828125
iteration: 1
render loss: -0.271240234375
iteration: 2
render loss: -0.27978515625
iteration: 3
render loss: -0.28564453125
iteration: 4
render loss: -0.29052734375
iteration: 5
render loss: -0.29296875
iteration: 6
render loss: -0.298828125
iteration: 7
render loss: -0.288330078125
iteration: 8
render loss: -0.30126953125
iteration: 9
render loss: -0.296630859375
iteration: 10
render loss: -0.29833984375
iteration: 11
render loss: -0.29345703125
iteration: 12
render loss: -0.305419921875
iteration: 13
render loss: -0.310302734375
iteration: 14
render loss: -0.31103515625
iteration: 15
render loss: -0.31787109375
iteration: 16
render loss: -0.29931640625
iteration: 17
render loss: -0.3056640625
iteration: 18
render loss: -0.31689453125
iteration: 19
render loss: -0.326171875
iteration: 20
render loss: -0.33056640625
iteration: 21
render loss: -0.322509765625
iteration: 22
render loss: -0.328369140625
iteration: 23
rend

render loss: -0.3271484375
iteration: 194
render loss: -0.338623046875
iteration: 195
render loss: -0.34326171875
iteration: 196
render loss: -0.33642578125
iteration: 197
render loss: -0.34033203125
iteration: 198
render loss: -0.33984375
iteration: 199
render loss: -0.340576171875
iteration: 200
render loss: -0.343994140625
iteration: 201
render loss: -0.3349609375
iteration: 202
render loss: -0.34716796875
iteration: 203
render loss: -0.34130859375
iteration: 204
render loss: -0.342041015625
iteration: 205
render loss: -0.34765625
iteration: 206
render loss: -0.34814453125
iteration: 207
render loss: -0.348388671875
iteration: 208
render loss: -0.344482421875
iteration: 209
render loss: -0.347900390625
iteration: 210
render loss: -0.346923828125
iteration: 211
render loss: -0.352783203125
iteration: 212
render loss: -0.3486328125
iteration: 213
render loss: -0.350341796875
iteration: 214
render loss: -0.35693359375
iteration: 215
render loss: -0.353759765625
iteration: 216
render lo

KeyboardInterrupt: 

## Render spinning view

In [8]:
from math import *

frame = 0

for theta in np.arange(0,2*3.1415,0.05):
    with torch.no_grad():
        cam_pos = torch.cuda.FloatTensor(np.array([3*cos(theta), 0.2, 3*sin(theta)]))
        cam_targ = torch.zeros(3).to(device)
    
        img = render(cam_pos, cam_targ, strokes)

        # Save the intermediate render.
        im = Image.fromarray(np.clip(255*img.cpu().detach().numpy(),0,255).astype(np.uint8))
        im.save("output/%.6d.png" % frame)
        
        frame += 1


## Save strokes for other applications/re-rendering

In [8]:
output_filename = "a_bunch_of_strokes.pth"

torch.save(strokes, open(output_filename,"wb"))